#### Data: https://public.tableau.com/s/sites/default/files/media/faa_data_subset.xlsx

#### we are looking to explore hdf5 with wildlife strikes dataset

In [1]:
import pandas as pd
import h5py

In [2]:
filename = 'faa_data_subset.xlsx'

In [3]:
# read the xls file with the sheetname
sheets = pd.ExcelFile(filename)
sheets.sheet_names

[u'FAA Wildlife Strikes']

In [4]:
# look at top 3 rows
df = pd.read_excel(filename, sheet=sheets.sheet_names[0])
df.head(3)

,Airport: Code,Airport: Name,Origin State,Origin State Code,Country,Aircraft: Type,Aircraft: Number of engines,Collision Date and Time,When: Time of day,When: Phase of flight,...,Days,Feet above ground,Miles from airport,Wildlife: Animal Category,Wildlife: Species Order,Wildlife: Species Group,Wildlife: Species,Wildlife: Species ID,Number of Strikes,Record ID
0,KAAF,APALACHICOLA REGIONAL ARPT,Florida,FL,United States,NaN,NaN,2012-09-20 19:30:00,NaN,Take-off run,...,NaN,0.0,NaN,Terrestrial Mammals,Canids,"Wolves, Dogs, Foxes, Coyote",Domestic dog,1F12,1,17459
1,KAAF,APALACHICOLA REGIONAL ARPT,Florida,FL,United States,Airplane,1.0,2013-04-23 17:09:00,NaN,Take-off run,...,NaN,NaN,NaN,Birds,"Pelicans, Herons, Egrets, Bitterns, Ibises","Herons, Egrets, Bitterns","Herons, egrets, bitterns",I1,1,17114
2,KABE,LEHIGH VALLEY INTL,Pennsylvania,PA,United States,Airplane,2.0,2009-04-23 09:22:00,Day,Take-off run,...,30.0,0.0,0.0,Birds,"Caracaras, Falcons","Caracaras, Falcons",American kestrel,K5114,1,259361


In [5]:
# look at all the columns, as there are 25 columns we will be able to drop some columns which we don't require
df.columns

Index([u'Airport: Code', u'Airport: Name', u'Origin State',
       u'Origin State Code', u'Country', u'Aircraft: Type',
       u'Aircraft: Number of engines', u'Collision Date and Time',
       u'When: Time of day', u'When: Phase of flight',
       u'Effect: Amount of damage (detailed)', u'Effect: Impact to flight',
       u'Effect: Indicated Damage',
       u'Cost: Aircraft time out of service (hours)', u'Cost: Total $',
       u'Days', u'Feet above ground', u'Miles from airport',
       u'Wildlife: Animal Category', u'Wildlife: Species Order',
       u'Wildlife: Species Group', u'Wildlife: Species',
       u'Wildlife: Species ID', u'Number of Strikes', u'Record ID'],
      dtype='object')

In [6]:
# time the file read 
%timeit pd.read_excel(filename, sheet=sheets.sheet_names[0])

1 loop, best of 3: 12.9 s per loop


In [7]:
# look at the length of rows
len(df)

28298

##### 8 seconds it take to read the file

In [8]:
# 'Airport: Code', 'Origin State Code', 'Wildlife: Species ID', 'Record ID' columns is not needed
for column in ['Airport: Code', 'Origin State Code', 'Wildlife: Species ID', 'Record ID']:
    if column not in ['Record ID']:
        print ", ".join(df[column].values.tolist()[:10])
    else:
        print df[column].values.tolist()[:10]

KAAF, KAAF, KABE, KABE, KABE, KABE, KABE, KABE, KABE, KABE
FL, FL, PA, PA, PA, PA, PA, PA, PA, PA
1F12, I1, K5114, O2205, K3302, NE1, K3302, YM1102, O2205, K3302
[17459L, 17114L, 259361L, 345167L, 262782L, 208167L, 344633L, 4255L, 9581L, 1761L]


In [9]:
# not sure about days column too, so we can ignore this too
df['Days'].value_counts()[:1]

0.000000    653
0.041667    639
0.083333    212
1.000000    150
Name: Days, dtype: int64

In [10]:
df.columns = [x.lower() for x in df.columns]
df.columns = [x.replace(' ','_') for x in df.columns]

In [11]:
df.columns

Index([u'airport:_code', u'airport:_name', u'origin_state',
       u'origin_state_code', u'country', u'aircraft:_type',
       u'aircraft:_number_of_engines', u'collision_date_and_time',
       u'when:_time_of_day', u'when:_phase_of_flight',
       u'effect:_amount_of_damage_(detailed)', u'effect:_impact_to_flight',
       u'effect:_indicated_damage',
       u'cost:_aircraft_time_out_of_service_(hours)', u'cost:_total_$',
       u'days', u'feet_above_ground', u'miles_from_airport',
       u'wildlife:_animal_category', u'wildlife:_species_order',
       u'wildlife:_species_group', u'wildlife:_species',
       u'wildlife:_species_id', u'number_of_strikes', u'record_id'],
      dtype='object')

In [12]:
exclude_columns = ['airport:_code', 'origin_state_code', 'wildlife:_species id', 'record_id', 'days']
include_columns = []
for column in df.columns:
    if column not in exclude_columns:
        include_columns.append(column)

In [13]:
map_str = ['country', 'origin_state']
df[map_str] = df[map_str].astype('str')

In [14]:
", ".join(include_columns)

u'airport:_name, origin_state, country, aircraft:_type, aircraft:_number_of_engines, collision_date_and_time, when:_time_of_day, when:_phase_of_flight, effect:_amount_of_damage_(detailed), effect:_impact_to_flight, effect:_indicated_damage, cost:_aircraft_time_out_of_service_(hours), cost:_total_$, feet_above_ground, miles_from_airport, wildlife:_animal_category, wildlife:_species_order, wildlife:_species_group, wildlife:_species, wildlife:_species_id, number_of_strikes'

In [15]:
df[include_columns].head(5)

,airport:_name,origin_state,country,aircraft:_type,aircraft:_number_of_engines,collision_date_and_time,when:_time_of_day,when:_phase_of_flight,effect:_amount_of_damage_(detailed),effect:_impact_to_flight,...,cost:_aircraft_time_out_of_service_(hours),cost:_total_$,feet_above_ground,miles_from_airport,wildlife:_animal_category,wildlife:_species_order,wildlife:_species_group,wildlife:_species,wildlife:_species_id,number_of_strikes
0,APALACHICOLA REGIONAL ARPT,Florida,United States,NaN,NaN,2012-09-20 19:30:00,NaN,Take-off run,None,None,...,NaN,0,0.0,NaN,Terrestrial Mammals,Canids,"Wolves, Dogs, Foxes, Coyote",Domestic dog,1F12,1
1,APALACHICOLA REGIONAL ARPT,Florida,United States,Airplane,1.0,2013-04-23 17:09:00,NaN,Take-off run,None,None,...,NaN,0,NaN,NaN,Birds,"Pelicans, Herons, Egrets, Bitterns, Ibises","Herons, Egrets, Bitterns","Herons, egrets, bitterns",I1,1
2,LEHIGH VALLEY INTL,Pennsylvania,United States,Airplane,2.0,2009-04-23 09:22:00,Day,Take-off run,Medium,Aborted Take-off,...,720.0,171132,0.0,0.0,Birds,"Caracaras, Falcons","Caracaras, Falcons",American kestrel,K5114,1
3,LEHIGH VALLEY INTL,Pennsylvania,United States,Airplane,2.0,2014-04-13 22:00:00,Night,Approach,None,None,...,15.0,600,NaN,NaN,Birds,Pigeons and Doves,Doves,Mourning dove,O2205,1
4,LEHIGH VALLEY INTL,Pennsylvania,United States,Airplane,2.0,2009-03-31 18:15:00,Day,Approach,Medium,None,...,12.0,188245,NaN,0.0,Birds,"Hawks, Kites, Eagles, Ospreys, Vultures","Kites, Hawks, Eagles",Red-tailed hawk,K3302,1


#### creating multiple hdf files with different view levels will help to reduce the load time

In [16]:
# agg by Origin State, Country, Number of Strikes
state_country_view = df.copy()
# renaming columns which has spaces, replacing spaces with underscores
# converting object columns to str
state_country_view = state_country_view.groupby(['country', 'origin_state']).agg({'number_of_strikes': 'sum'})
state_country_view = state_country_view.sort_values('number_of_strikes', ascending=False)

In [17]:
state_country_view.to_hdf('data_views.h5', key='state_country_view', mode='a')

In [18]:
%timeit pd.read_hdf('data_views.h5', key='state_country_view', mode='r')

100 loops, best of 3: 10.6 ms per loop


In [19]:
state_country_df = pd.read_hdf('data_views.h5', key='state_country_view', mode='r')

In [20]:
state_country_df.head(5)

number_of_strikes
country       origin_state                   
United States California                 3026
              Texas                      2306
              Florida                    2239
              New York                   2140
              Pennsylvania               1286

In [21]:
state_country_df.loc[('United States', ['North Dakota', 'Kansas', 'Montana']), :]

number_of_strikes
country       origin_state                   
United States North Dakota                191
              Kansas                      153
              Montana                      89

#### hdf5 preserves the index

In [22]:
# agg by Origin State, Collision Date and Time, Country, Number of Strikes 
datewise_view = df.copy()
datewise_view['collision_date_and_time'] = pd.to_datetime(datewise_view['collision_date_and_time'])
datewise_view['collision_date_and_time'] = datewise_view['collision_date_and_time'].apply(lambda v: v.date())
datewise_view = datewise_view.groupby(['country', 'collision_date_and_time', 'origin_state']).agg({'number_of_strikes': 'sum'})

In [23]:
datewise_view

number_of_strikes
country       collision_date_and_time origin_state                     
United States 2000-01-02              California                      1
              2000-01-03              Florida                         1
                                      Rhode Island                    1
              2000-01-04              New Jersey                      1
              2000-01-05              Connecticut                     1
                                      Ohio                            1
              2000-01-06              New York                        1
              2000-01-08              Colorado                        1
                                      Florida                         1
                                      New York                        2
              2000-01-09              Indiana                         1
              2000-01-10              Ohio                            1
              2000-01-11              California                      1
              2000-01-12              California                      2
                                      New York                        1
              2000-01-14              Connecticut                     1
              2000-01-18              California                      1
              2000-01-19              Connecticut                     1
              2000-01-20              California                      1
                                      Oregon                          2
              2000-01-22              New Jersey                      1
              2000-01-23              New York                        1
              2000-01-24              Texas                           1
              2000-01-26              Arizona                         1
              2000-01-27              California                      1
              2000-01-30              Rhode Island                    1
                                      Tennessee                       1
              2000-02-01              Missouri                        1
              2000-02-02              New York                        1
              2000-02-05              North Carolina                  1
...                                                                 ...
              2015-05-27              Iowa                            1
                                      North Carolina                  1
                                      Ohio                            1
                                      Texas                           1
              2015-05-28              Alabama                         1
                                      California                      2
                                      Florida                         1
                                      Maryland                        1
                                      North Carolina                  1
                                      Oregon                          1
                                      Rhode Island                    1
                                      Texas                           1
              2015-05-29              California                      2
                                      Colorado                        1
                                      Missouri                        1
                                      New York                        1
                                      North Dakota                    1
                                      Oklahoma                        1
                                      Texas                           1
                                      Wisconsin                       2
              2015-05-30              Colorado                        1
                                      North Dakota                    1
                                      Oklahoma                        1
                      

In [24]:
datewise_view.to_hdf('data_views.h5', key='datewise_view', mode='a')

In [25]:
pd.read_hdf('data_views.h5', key='state_country_view', mode='r').head(5)

number_of_strikes
country       origin_state                   
United States California                 3026
              Texas                      2306
              Florida                    2239
              New York                   2140
              Pennsylvania               1286

In [26]:
pd.read_hdf('data_views.h5', key='datewise_view', mode='r').head(5)

number_of_strikes
country       collision_date_and_time origin_state                   
United States 2000-01-02              California                    1
              2000-01-03              Florida                       1
                                      Rhode Island                  1
              2000-01-04              New Jersey                    1
              2000-01-05              Connecticut                   1

#### hdf5 preserves the datetime format, which avoids the date column to be parsed again and also as the date is in index it allows for faster data filtering
#### below we are selecting a particular date and range of dates

In [27]:
datewise_view = pd.read_hdf('data_views.h5', key='datewise_view', mode='r')

In [28]:
strt_dte = pd.to_datetime('2000-01-02').date()
datewise_view.loc[('United States', strt_dte), :]

,number_of_strikes
origin_state,
California,1


In [29]:
datewise_view[(datewise_view.index.get_level_values('collision_date_and_time') >= pd.to_datetime('2010-01-01').date()) 
              & (datewise_view.index.get_level_values('collision_date_and_time') <= pd.to_datetime('2010-02-01').date()) ]

number_of_strikes
country       collision_date_and_time origin_state                   
United States 2010-01-01              Connecticut                   1
                                      Florida                       3
                                      Texas                         1
              2010-01-02              Florida                       1
                                      Maryland                      1
                                      Utah                          1
              2010-01-03              California                    4
                                      New York                      1
                                      Oregon                        1
              2010-01-04              Michigan                      1
              2010-01-05              California                    5
                                      Washington                    1
              2010-01-06              California                    1
              2010-01-07              Florida                       1
              2010-01-08              California                    1
                                      New York                      1
                                      Texas                         1
                                      Utah                          1
              2010-01-09              New York                      1
              2010-01-10              California                    4
                                      Kansas                        1
                                      Missouri                      1
                                      Utah                          1
                                      Virginia                      1
              2010-01-11              California                    1
                                      Missouri                      1
                                      Texas                         1
                                      Virginia                      1
              2010-01-12              Connecticut                   1
                                      Texas                         1
...                                                               ...
              2010-01-22              DC                            1
                                      Ohio                          1
                                      Utah                          1
              2010-01-24              California                    1
                                      Connecticut                   1
                                      Florida                       1
              2010-01-25              California                    1
                                      Florida                       1
                                      New Mexico                    1
              2010-01-26              California                    1
                                      DC                            1
                                      Maryland                      1
                                      Missouri                      2
                                      New York                      1
              2010-01-27              Alabama                       1
                                      California                    1
                                      Florida                       3
                                      Louisiana                     1
              2010-01-28              New York                      1
                                      Texas                         1
                                      Utah                          1
              2010-01-29              California                    1
              2010-01-30              California                    2
                                      Florida                       1
                                      Louisiana               

In [30]:
#todo: create more views to reduce time in aggrs